In [62]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from pathlib import Path
from keras.utils.np_utils import to_categorical

In [63]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [64]:
#normalize data
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
x_train/=255.0
x_test/=255.0


In [65]:
#convert class vectors to binary class matrices
y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)

In [66]:
#create model and add layers
model=Sequential()
model.add(Conv2D(32,(3,3),padding='same',input_shape=(32,32,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

In [67]:
#compile the model
model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)

In [68]:
#print the summary for the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0

In [16]:
#train the model
model.fit(
   x_train,
   y_train,
   batch_size=32,
   epochs=20,
   validation_data=(x_test,y_test),
   shuffle=True
)

Epoch 1/20
1563/1563 [==============================] - 166s 106ms/step - loss: 1.5079 - accuracy: 0.4512 - val_loss: 1.1232 - val_accuracy: 0.6010
Epoch 2/20
1563/1563 [==============================] - 193s 124ms/step - loss: 1.1233 - accuracy: 0.6015 - val_loss: 0.9455 - val_accuracy: 0.6693
Epoch 3/20
1563/1563 [==============================] - 174s 111ms/step - loss: 0.9716 - accuracy: 0.6568 - val_loss: 0.8581 - val_accuracy: 0.6975
Epoch 4/20
1563/1563 [==============================] - 175s 112ms/step - loss: 0.8785 - accuracy: 0.6917 - val_loss: 0.7925 - val_accuracy: 0.7214
Epoch 5/20
1563/1563 [==============================] - 176s 112ms/step - loss: 0.8186 - accuracy: 0.7131 - val_loss: 0.7459 - val_accuracy: 0.7380
Epoch 6/20
1563/1563 [==============================] - 175s 112ms/step - loss: 0.7621 - accuracy: 0.7348 - val_loss: 0.7306 - val_accuracy: 0.7475
Epoch 7/20
1563/1563 [==============================] - 176s 112ms/step - loss: 0.7253 - accuracy: 0.7436 - val_

In [69]:
#save the neural network structure
model_structure=model.to_json()
f=Path("model_structure.json")
f.write_text(model_structure)

4383

In [70]:
#save the trained neural network weights
model.save_weights("model_weights.h5")

In [71]:
#make predictions on images
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np

In [72]:
#cifar-10 dataset class labels
class_labels=[
    "Planes",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Boat",
    "Truck",
]

In [73]:
#load the json file that contains model structure
f=Path("model_structure.json")
model_structure=f.read_text()

In [74]:
#recreate the keras model object from the json data
model=model_from_json(model_structure)

In [75]:
#reload the model trained weights
model.load_weights("model_weights.h5")

In [76]:
!pip install Pillow

In [77]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array


In [89]:
#load an image pil to test
img=load_img("truck.jpg",target_size=(32,32))

In [79]:
from keras.models import load_model

In [80]:
#convert the image to a numpy array
image_to_test=img_to_array(img)

In [81]:
import numpy as np

In [82]:
#add a 4th dimension to image
list_of_images=np.expand_dims(image_to_test,axis=0)

In [83]:
#make predictions using model
results=model.predict(list_of_images)

In [84]:
#1 testing image,check first result
single_result=results[0]

In [85]:
#we will get a likelihood score for all 10 possible classes,findout the highest score
most_likely_class_index=int(np.argmax(single_result))
class_likelihood=single_result[most_likely_class_index]

In [86]:
#get the name of the most likely class
class_label=class_labels[most_likely_class_index]

In [90]:
#print the result
print("This is image is a {}-Likelihood:{:2f}".format(class_label,class_likelihood))

This is image is a Truck-Likelihood:0.998263
